<a href="https://colab.research.google.com/github/spanman888/DSC672_Teacher_Salary/blob/master/TeacherSalaryModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import os
import numpy as np
import math
import time
import matplotlib.pyplot as plt
import sys
import csv
from operator import itemgetter
import copy

from sklearn.model_selection import cross_validate, train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.externals import joblib
from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif, chi2
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import KBinsDiscretizer, scale

pd.set_option('display.max_columns', 100)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
# In order to download files to local file system
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [0]:
# download numeric dataframe
df_num = pd.read_csv('/content/drive/My Drive/df_num.csv', index_col=0)
df_num.head(10)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,AdjSalMonth,months_employed,fte,dist_exp,state_exp,out_of_state_exp,pct_admin,first_3,multi_assign_2,multi_assign_3,grade_taught,Female,Male,Bachelors,Doctorate,Masters,Edu_None,Edu_Other,AmerIn_Alaska,Asian_Pacific,Black,Hispanic,Eth_unknown,White,Elem_Teacher,HS_Teacher,JuniorHS_Teacher,Kinder_Teacher,PreK_Teacher,SpecEd_Teacher,Multi_Taught,OneSch_Taught,Chicago,Other,Suburb,Lit/Language,Humanities/SS,Math/Science,Art/PE,SpecialEd,Misc.
0,4029.489540,9,100,1.0,1.0,0.0,0.0,601,0,0,7,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0
1,12889.019160,9,100,23.0,27.0,1.0,0.0,600,0,0,8,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1
3,8668.664381,9,100,12.0,12.0,0.0,0.0,601,0,0,8,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1
4,10681.660375,10,100,5.0,5.0,8.0,0.0,605,0,0,12,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0
5,8012.189620,10,100,1.0,2.0,9.0,0.0,600,0,0,5,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
6,7298.253984,9,100,9.0,9.0,0.0,0.0,600,0,0,6,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1
7,8576.624386,9,100,8.0,8.0,0.0,0.0,606,0,0,12,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0
8,7031.448365,10,100,3.0,3.0,0.0,0.0,601,1,1,6,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0
9,7751.625187,9,100,2.0,2.0,0.0,0.0,605,0,0,12,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0
10,6745.656485,10,100,3.0,3.0,0.0,0.0,602,0,0,12,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0


In [0]:
df_num.shape

(1301569, 41)

In [0]:
# download categorical dataframe
df_cat = pd.read_csv('drive/My Drive/df_cat.csv', index_col=0)
df_cat.head(10)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,AdjSalMonth,sch_num,gender,emply_desc,months_employed,fte,dist_exp,state_exp,out_of_state_exp,pos_cd,low_grade,high_grade,pct_admin,first_3,Zip_Type,multi_assign_2,multi_assign_3,eth_groups,edu_groups,grade_taught,loc_taught,assign1_groups
0,4029.489540,1003,F,Full-Time,9,100,1.0,1.0,0.0,20,00,07,0.0,601,Suburb,0,0,White,Bachelors,7,OneSch_Taught,3
1,12889.019160,1005,F,Full-Time,9,100,23.0,27.0,1.0,20,07,08,0.0,600,Suburb,0,0,White,Masters,8,OneSch_Taught,6
3,8668.664381,1003,F,Full-Time,9,100,12.0,12.0,0.0,20,07,08,0.0,601,Suburb,0,0,White,Masters,8,OneSch_Taught,6
4,10681.660375,2,F,Full-Time,10,100,5.0,5.0,8.0,22,09,12,0.0,605,Suburb,0,0,White,Masters,12,OneSch_Taught,1
5,8012.189620,2002,F,Full-Time,10,100,1.0,2.0,9.0,19,01,05,0.0,600,Suburb,0,0,White,Doctorate,5,OneSch_Taught,4
6,7298.253984,2002,F,Full-Time,9,100,9.0,9.0,0.0,20,00,06,0.0,600,Suburb,0,0,White,Masters,6,OneSch_Taught,6
7,8576.624386,30,F,Full-Time,9,100,8.0,8.0,0.0,22,09,12,0.0,606,Chicago,0,0,White,Bachelors,12,OneSch_Taught,4
8,7031.448365,1017,M,Full-Time,10,100,3.0,3.0,0.0,20,00,06,0.0,601,Suburb,1,1,White,Masters,6,OneSch_Taught,1
9,7751.625187,13,M,Full-Time,9,100,2.0,2.0,0.0,22,09,12,0.0,605,Suburb,0,0,White,Masters,12,OneSch_Taught,3
10,6745.656485,1,F,Full-Time,10,100,3.0,3.0,0.0,22,09,12,0.0,602,Other,0,0,White,Masters,12,OneSch_Taught,2


In [0]:
df_cat.shape

(1301569, 22)

**FEATURE SELECTION**

---
Methods:
- Low Variance
- Stepwise Recursive Backwards Feature Removal
- Wrapper Select
- Univariate Feature Selection - Mutual Info Regression
- Full Blown Wrapper Select (Any type of model)

In [0]:
### VARIABLES FOR FEATURE SELECTION ###

feat_select = 1     # Feature Selection ON - 1 or OFF - 0                                                                                        
fs_type = 3         # Feature Selection TYPE (1 = Stepwise Backwards Removal, 2 = Wrapper Select, 3 = Univariate Selection, 4 = Full Blown Wrapper)
lv_filter = 1       # Low Variance FIlter ON - 1 or OFF - 0
feat_start = 1      # Feature Start Column 
k_cnt = 5           # Number of 'TOP' Ranking Features

rand_st = 1
data = []
target = []

In [0]:
header = []
for q in df_num:
  header.append(q)

for x in df_num.itertuples(index=False):
  if x[0] == '':
    continue
  else:
    target.append(x[0])
  
  temp=[]
  for i in range(feat_start, len(df_num.columns)):
    if x[i] == '':
      continue
    else:
      temp.append(float(x[i]))
                  
  data.append(temp)
 
#Test Print
print(len(target),len(data))
print('\n')

data_np=np.asarray(data)
target_np=np.asarray(target)

1295444 1295444




In [0]:
### LOW VARIANCE FILTER ###
if lv_filter==1:
  print('### LOW VARIANCE FILTER is ON ###', '\n')
    
  #LV Threshold
  sel = VarianceThreshold(threshold=0.5)                                      #Removes any feature with less than 20% variance
  fit_mod=sel.fit(data_np)
  fitted=sel.transform(data_np)
  sel_idx=fit_mod.get_support()

  #Get lists of selected and non-selected features (names and indexes)
  temp=[]
  temp_idx=[]
  temp_del=[]
  for i in range(len(data_np[0])):
    if sel_idx[i]==1:                                                           #Selected Features get added to temp header
      temp.append(header[i+feat_start])
      temp_idx.append(i)
    else:                                                                       #Indexes of non-selected features get added to delete arra
      temp_del.append(i)

  print('Selected', temp)
  print('Features (total, selected):', len(data_np[0]), len(temp))
  print('\n')
    

### LOW VARIANCE FILTER is ON ### 

Selected ['fte', 'dist_exp', 'state_exp', 'out_of_state_exp', 'pct_admin', 'first_3', 'grade_taught']
Features (total, selected): 40 7




In [0]:
### FEATURE SELECTION METHODS ###

if feat_select == 1:
  print('### FEATURE SELECTION is ON ###', '\n')
  
  ### STEPWISE RECURSIVE BACKWARDS ###
  if fs_type == 1:
    print('Stepwise Recursive Backwards - Random Forest: ')
    rgr = RandomForestRegressor(n_estimators=500, max_depth=None, min_samples_split=3, criterion='mse', random_state=rand_st)
    sel = RFE(rgr, n_features_to_select=k_cnt, step=.1)
      
    fit_mod=sel.fit(data_np, target_np)
    print(sel.ranking_)
    sel_idx=fit_mod.get_support()  
    
  ### WRAPPER SELECT ###
  if fs_type == 2:
    print ('Wrapper Select: ')
    rgr= SVR(kernel='linear', gamma=0.1, C=1.0)
    sel = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
    
    fit_mod=sel.fit(data_np, target_np)    
    sel_idx=fit_mod.get_support()
    
  ### UNIVARIATE FEATURE SELECTION ###
  if fs_type == 3:
    print ('Univariate Feature Selection - Mutual Info: ')
    sel=SelectKBest(mutual_info_regression, k=k_cnt)
    fit_mod=sel.fit(data, target)
    sel_idx=fit_mod.get_support()

    #Print ranked variables out sorted
    temp=[]
    scores=fit_mod.scores_
    for i in range(feat_start, len(header)):
      temp.append([header[i], float(scores[i-feat_start])])
      print('Ranked Features')
      temp_sort=sorted(temp, key=itemgetter(1), reverse=True)
      for i in range(len(temp_sort)):
        print(i, temp_sort[i][0], ':', temp_sort[i][1])
      print('\n')
    
    ### FULL BLOWN WRAPPER ###
    if fs_type == 4:
      start_ts=time.time()
      sel_idx=[]                                                                                      #Empty array to hold optimal selected feature set
      best_score=sys.float_info.max                                                                   #For regression compare RMSE, lower is better, so start with max sys float value
      feat_cnt=len(data[0])

      #Create Wrapper model
      rgr= SVR(kernel='linear', gamma=0.1, C=1.0)                    #This could be any kind of regressor model         
        
      #Loop thru feature sets
      roll_idx=0
      combo_ctr=0
      feat_arr=[0 for col in range(feat_cnt)]                                         #Initialize feature array
      for idx in range(feat_cnt):
        roll_idx=idx
        feat_space_search(feat_arr, idx)                                           #Recurse
        feat_arr=[0 for col in range(feat_cnt)]                                     #Reset feature array after each iteration
        
      print('# of Feature Combos Tested:', combo_ctr)
      print(best_score, sel_idx, len(data[0]))
      print("Wrapper Feat Sel Runtime:", time.time()-start_ts)
  
  # PRINT LIST OF SELECTED & NON-SELECTED FEATURES
  temp=[]
  temp_idx=[]
  temp_del=[]
  for i in range(len(data_np[0])):
      if sel_idx[i]==1:                                                           #Selected Features get added to temp header
          temp.append(header[i+feat_start])
          temp_idx.append(i)
      else:                                                                       #Indexes of non-selected features get added to delete array
          temp_del.append(i)
  print('Selected', temp)
  print('Features (total/selected):', len(data_np[0]), len(temp))
  print('\n')
    
  # PRINT LIST OF SELECTED & NON-SELECTED FEATURES
  #header = header[0:feat_start]
  #for field in temp:
  #     header.append(field)
  #data_np = np.delete(data_np, temp_del, axis=1) 

### FEATURE SELECTION is ON ### 

Univariate Feature Selection - Mutual Info: 


In [0]:
#create new data frames based on FS results - 5, 7, or 8 features
df_fs5 = df_num.filter(['AdjSalMonth','fte','dist_exp','state_exp','first_3','Bachelors'], axis=1)
df_fs7 = df_num.filter(['AdjSalMonth','months_employed','fte','dist_exp','state_exp','first_3','grade_taught','Bachelors'], axis=1)
df_fs8 = df_num.filter(['AdjSalMonth','months_employed','dist_exp','state_exp','out_of_state_exp','pct_admin','first_3','grade_taught'], axis=1)

In [0]:
df_fs5.head(5)

,AdjSalMonth,fte,dist_exp,state_exp,first_3,Bachelors
0,4029.489540,100,1.0,1.0,601,1
1,12889.019160,100,23.0,27.0,600,0
3,8668.664381,100,12.0,12.0,601,0
4,10681.660375,100,5.0,5.0,605,0
5,8012.189620,100,1.0,2.0,600,0


In [0]:
df_fs7.head(5)

,AdjSalMonth,months_employed,fte,dist_exp,state_exp,first_3,grade_taught,Bachelors
0,4029.489540,9,100,1.0,1.0,601,7,1
1,12889.019160,9,100,23.0,27.0,600,8,0
3,8668.664381,9,100,12.0,12.0,601,8,0
4,10681.660375,10,100,5.0,5.0,605,12,0
5,8012.189620,10,100,1.0,2.0,600,5,0


In [0]:
df_fs8.head(5)

,AdjSalMonth,months_employed,dist_exp,state_exp,out_of_state_exp,pct_admin,first_3,grade_taught
0,4029.489540,9,1.0,1.0,0.0,0.0,601,7
1,12889.019160,9,23.0,27.0,1.0,0.0,600,8
3,8668.664381,9,12.0,12.0,0.0,0.0,601,8
4,10681.660375,10,5.0,5.0,8.0,0.0,605,12
5,8012.189620,10,1.0,2.0,9.0,0.0,600,5


**DATA SPLIT**

---



In [0]:
### TEST TRAIN VALIDATION SPLIT on TOTAL ###

num_instances = df_num.shape[0]

Y = df_num['AdjSalMonth']
X = df_num.drop(['AdjSalMonth'], axis=1)

# Creating the train/test split
Y_train, Y_test, X_train, X_test = train_test_split(Y, X, test_size=0.3, random_state=33)  

# Splitting df_train into the train and then holding some out for validation
Y_train, Y_val, X_train, X_val = train_test_split(Y_train, X_train, test_size=0.143, random_state=33)

print("Confirming Splits...")
print("Y_TRAIN: " + str(Y_train.shape[0]) + " = " + str(round((Y_train.shape[0]/num_instances)*100, 2)) + "%")
print("Y_TEST: " + str(Y_test.shape[0]) + " = " + str(round((Y_test.shape[0]/num_instances)*100, 2)) + "%")
print("Y_VALIDATION: " + str(Y_val.shape[0]) + " = " + str(round((Y_val.shape[0]/num_instances)*100, 2)) + "%")

print()
print("X_TRAIN: " + str(X_train.shape[0]) +  ", " + str(X_train.shape[1]) + " = " + str(round((X_train.shape[0]/num_instances)*100, 2)) + "%")
print("X_TEST: " + str(X_test.shape[0]) +  ", " + str(X_test.shape[1]) + " = " + str(round((X_test.shape[0]/num_instances)*100, 2)) + "%")
print("X_VALIDATION: " + str(X_val.shape[0]) +  ", " + str(X_val.shape[1]) + " = " + str(round((X_val.shape[0]/num_instances)*100, 2)) + "%")

# This is a *close* Train/Test/Validation split of 60/30/10

# X_train, Y_train is the data set we should train all the models with
# X_test, Y_test is the data we will be testing our models on to assess accuracy
# X_val, Y_val is the validation data set 

Confirming Splits...
Y_TRAIN: 780810 = 59.99%
Y_TEST: 390471 = 30.0%
Y_VALIDATION: 130288 = 10.01%

X_TRAIN: 780810, 40 = 59.99%
X_TEST: 390471, 40 = 30.0%
X_VALIDATION: 130288, 40 = 10.01%


In [0]:
### TEST TRAIN VALIDATION SPLIT on 5 Feature Dataset ###

num_instances = df_fs5.shape[0]

Y_fs5 = df_fs5['AdjSalMonth']
X_fs5 = df_fs5.drop(['AdjSalMonth'], axis=1)

# Creating the train/test split
Y_train_fs5, Y_test_fs5, X_train_fs5, X_test_fs5 = train_test_split(Y_fs5, X_fs5, test_size=0.3, random_state=33)  

# Splitting df_train into the train and then holding some out for validation
Y_train_fs5, Y_val_fs5, X_train_fs5, X_val_fs5 = train_test_split(Y_train_fs5, X_train_fs5, test_size=0.143, random_state=33)

print("Confirming Splits...")
print("Y_TRAIN: " + str(Y_train_fs5.shape[0]) + " = " + str(round((Y_train_fs5.shape[0]/num_instances)*100, 2)) + "%")
print("Y_TEST: " + str(Y_test_fs5.shape[0]) + " = " + str(round((Y_test_fs5.shape[0]/num_instances)*100, 2)) + "%")
print("Y_VALIDATION: " + str(Y_val_fs5.shape[0]) + " = " + str(round((Y_val_fs5.shape[0]/num_instances)*100, 2)) + "%")

print()
print("X_TRAIN: " + str(X_train_fs5.shape[0]) +  ", " + str(X_train_fs5.shape[1]) + " = " + str(round((X_train_fs5.shape[0]/num_instances)*100, 2)) + "%")
print("X_TEST: " + str(X_test_fs5.shape[0]) +  ", " + str(X_test_fs5.shape[1]) + " = " + str(round((X_test_fs5.shape[0]/num_instances)*100, 2)) + "%")
print("X_VALIDATION: " + str(X_val_fs5.shape[0]) +  ", " + str(X_val_fs5.shape[1]) + " = " + str(round((X_val_fs5.shape[0]/num_instances)*100, 2)) + "%")

# This is a *close* Train/Test/Validation split of 60/30/10

# X_train, Y_train is the data set we should train all the models with
# X_test, Y_test is the data we will be testing our models on to assess accuracy
# X_val, Y_val is the validation data set 

Confirming Splits...
Y_TRAIN: 780810 = 59.99%
Y_TEST: 390471 = 30.0%
Y_VALIDATION: 130288 = 10.01%

X_TRAIN: 780810, 5 = 59.99%
X_TEST: 390471, 5 = 30.0%
X_VALIDATION: 130288, 5 = 10.01%


In [0]:
### TEST TRAIN VALIDATION SPLIT on 7 Feature Dataset ###

num_instances = df_fs5.shape[0]

Y_fs7 = df_fs7['AdjSalMonth']
X_fs7 = df_fs7.drop(['AdjSalMonth'], axis=1)

# Creating the train/test split
Y_train_fs7, Y_test_fs7, X_train_fs7, X_test_fs7 = train_test_split(Y_fs7, X_fs7, test_size=0.3, random_state=33)  

# Splitting df_train into the train and then holding some out for validation
Y_train_fs7, Y_val_fs7, X_train_fs7, X_val_fs7 = train_test_split(Y_train_fs7, X_train_fs7, test_size=0.143, random_state=33)

print("Confirming Splits...")
print("Y_TRAIN: " + str(Y_train_fs7.shape[0]) + " = " + str(round((Y_train_fs7.shape[0]/num_instances)*100, 2)) + "%")
print("Y_TEST: " + str(Y_test_fs7.shape[0]) + " = " + str(round((Y_test_fs7.shape[0]/num_instances)*100, 2)) + "%")
print("Y_VALIDATION: " + str(Y_val_fs7.shape[0]) + " = " + str(round((Y_val_fs7.shape[0]/num_instances)*100, 2)) + "%")

print()
print("X_TRAIN: " + str(X_train_fs7.shape[0]) +  ", " + str(X_train_fs7.shape[1]) + " = " + str(round((X_train_fs7.shape[0]/num_instances)*100, 2)) + "%")
print("X_TEST: " + str(X_test_fs7.shape[0]) +  ", " + str(X_test_fs7.shape[1]) + " = " + str(round((X_test_fs7.shape[0]/num_instances)*100, 2)) + "%")
print("X_VALIDATION: " + str(X_val_fs7.shape[0]) +  ", " + str(X_val_fs7.shape[1]) + " = " + str(round((X_val_fs7.shape[0]/num_instances)*100, 2)) + "%")

# This is a *close* Train/Test/Validation split of 60/30/10

# X_train, Y_train is the data set we should train all the models with
# X_test, Y_test is the data we will be testing our models on to assess accuracy
# X_val, Y_val is the validation data set 

Confirming Splits...
Y_TRAIN: 780810 = 59.99%
Y_TEST: 390471 = 30.0%
Y_VALIDATION: 130288 = 10.01%

X_TRAIN: 780810, 7 = 59.99%
X_TEST: 390471, 7 = 30.0%
X_VALIDATION: 130288, 7 = 10.01%


In [0]:
### TEST TRAIN VALIDATION SPLIT on 8 Feature Dataset ###

num_instances = df_fs8.shape[0]

Y_fs8 = df_fs8['AdjSalMonth']
X_fs8 = df_fs8.drop(['AdjSalMonth'], axis=1)

# Creating the train/test split
Y_train_fs8, Y_test_fs8, X_train_fs8, X_test_fs8 = train_test_split(Y_fs8, X_fs8, test_size=0.3, random_state=33)  

# Splitting df_train into the train and then holding some out for validation
Y_train_fs8, Y_val_fs8, X_train_fs8, X_val_fs8 = train_test_split(Y_train_fs8, X_train_fs8, test_size=0.143, random_state=33)

print("Confirming Splits...")
print("Y_TRAIN: " + str(Y_train_fs8.shape[0]) + " = " + str(round((Y_train_fs8.shape[0]/num_instances)*100, 2)) + "%")
print("Y_TEST: " + str(Y_test_fs8.shape[0]) + " = " + str(round((Y_test_fs8.shape[0]/num_instances)*100, 2)) + "%")
print("Y_VALIDATION: " + str(Y_val_fs8.shape[0]) + " = " + str(round((Y_val_fs8.shape[0]/num_instances)*100, 2)) + "%")

print()
print("X_TRAIN: " + str(X_train_fs8.shape[0]) +  ", " + str(X_train_fs8.shape[1]) + " = " + str(round((X_train_fs8.shape[0]/num_instances)*100, 2)) + "%")
print("X_TEST: " + str(X_test_fs8.shape[0]) +  ", " + str(X_test_fs8.shape[1]) + " = " + str(round((X_test_fs8.shape[0]/num_instances)*100, 2)) + "%")
print("X_VALIDATION: " + str(X_val_fs8.shape[0]) +  ", " + str(X_val_fs8.shape[1]) + " = " + str(round((X_val_fs8.shape[0]/num_instances)*100, 2)) + "%")

# This is a *close* Train/Test/Validation split of 60/30/10

# X_train, Y_train is the data set we should train all the models with
# X_test, Y_test is the data we will be testing our models on to assess accuracy
# X_val, Y_val is the validation data set 

Confirming Splits...
Y_TRAIN: 780810 = 59.99%
Y_TEST: 390471 = 30.0%
Y_VALIDATION: 130288 = 10.01%

X_TRAIN: 780810, 7 = 59.99%
X_TEST: 390471, 7 = 30.0%
X_VALIDATION: 130288, 7 = 10.01%


_____________________________________________________

BEGINNING OF MODELS RUN WITH FULL FEATURES

In [0]:
### LINEAR REGRESSION ###

linear_model = LinearRegression()
linear_model.fit(X_train,Y_train)
r_squared = linear_model.score(X_train, Y_train)

print("Linear Regression Results\n")
print("COEFFICIENT OF DETERMINATION: ", round(r_squared, 5))

train_r_squared = linear_model.score(X_train, Y_train)
test_r_squared = linear_model.score(X_test, Y_test)
print("COEFFICIENT OF DETERMINATION - Train: ", train_r_squared)
print("COEFFICIENT OF DETERMINATION - Test: ", test_r_squared)

RMSE = math.sqrt(metrics.mean_squared_error(Y_train, linear_model.predict(X_train)))
print("\nLinear Regression RMSE: ", RMSE)
EV = metrics.explained_variance_score(Y_train, linear_model.predict(X_train))
print("Linear Regression Explained Variance: ", EV) 

Linear Regression Results

COEFFICIENT OF DETERMINATION:  0.58683
COEFFICIENT OF DETERMINATION - Train:  0.5868277780093627
COEFFICIENT OF DETERMINATION - Test:  0.5941099707824977

Linear Regression RMSE:  1700.20480712542
Linear Regression Explained Variance:  0.5868277780093628


In [0]:
# Plot output
predicted = linear_model.predict(X_test)

fig, ax = plt.subplots()
ax.scatter(Y_test, predicted, edgecolors=(0, 0, 0))
ax.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
plt.title('Predicted vs. Actual')
plt.show()

In [0]:
from yellowbrick.regressor import ResidualsPlot

# Instantiate the linear model and visualizer
linear = LinearRegression()
visualizer = ResidualsPlot(linear)

visualizer.fit(X_train, Y_train)  # Fit the training data to the model
visualizer.score(X_test, Y_test)  # Evaluate the model on the test data
visualizer.poof() 

In [0]:
### Decision Tree - All Depth###
from sklearn.tree import DecisionTreeRegressor
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train,Y_train)
r_squared = decision_tree.score(X_train, Y_train)

print("Decision Tree Results\n")
print("COEFFICIENT OF DETERMINATION: ", round(r_squared, 5))

train_r_squared = decision_tree.score(X_train, Y_train)
test_r_squared = decision_tree.score(X_test, Y_test)
print("COEFFICIENT OF DETERMINATION - Train: ", train_r_squared)
print("COEFFICIENT OF DETERMINATION - Test: ", test_r_squared)

RMSE = math.sqrt(metrics.mean_squared_error(Y_train, decision_tree.predict(X_train)))
print("\nDecision Tree RMSE: ", RMSE)
EV = metrics.explained_variance_score(Y_train, decision_tree.predict(X_train))
print("Decision Tree Explained Variance: ", EV) 

In [0]:
### Decision Tree - 3 Levels###
from sklearn.tree import DecisionTreeRegressor
rgr = DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=3, min_samples_split=2, min_samples_leaf=1, max_features=None)
rgr.fit(X_train,Y_train)
scores_r2 = rgr.score(X_train,Y_train)                                                                                                                          
print('Decision Tree R-Squared:', scores_r2)

In [0]:
from graphviz import Source
from sklearn import tree
from IPython.display import SVG
import time


graph = Source(tree.export_graphviz(rgr, out_file=None,filled = True,feature_names=X_train.columns.values))
display(SVG(graph.pipe(format='svg')))

In [0]:
import math
scorers = {'Neg_MSE': 'neg_mean_squared_error', 'expl_var': 'explained_variance'} 
    
#SciKit Decision Tree Regressor - Cross Val
start_ts=time.time()
rgr = DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=3, min_samples_split=2, min_samples_leaf=1, max_features=None) 
scores = cross_validate(rgr, X_train, y = Y_train, scoring = scorers, cv = 5)   

scores_RMSE = np.asarray([math.sqrt(-x) for x in scores['test_Neg_MSE']])                                       #Turns negative MSE scores into RMSE
scores_Expl_Var = scores['test_expl_var']
print("Decision Tree RMSE: %0.2f (+/- %0.2f)" % ((scores_RMSE.mean()), (scores_RMSE.std() * 2)))
print("Decision Tree Expl Var: %0.2f (+/- %0.2f)" % ((scores_Expl_Var.mean()), (scores_Expl_Var.std() * 2)))
print("CV Runtime:", time.time()-start_ts)

In [0]:
#Predict
rgr.fit(X_train,Y_train)
y_1 = rgr.predict(X_train)

In [0]:
#Plot of Residuals
plt.scatter(x=Y_train, y=y_1-Y_train)
plt.title('Decision Tree Residuals')
plt.ylabel('Residual (Predicted - Actual)')
plt.xlabel('Actual')
plt.show()

In [0]:
#normalizing data for KNN
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler().fit(X_train_fs7)
X_trainN_fs7 = min_max_scaler.transform(X_train_fs7)
X_testN_fs7 = min_max_scaler.transform(X_test_fs7)
X_valN_fs7 = min_max_scaler.transform(X_val_fs7)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [0]:
#KNN cont'd
import time
print('KNN Results')
print()
start_ts=time.time()
knn = KNeighborsRegressor(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, metric='euclidean', metric_params=None).fit(X_trainN_fs7, Y_train_fs7)

train_r2_salKNN = knn.score(X_trainN_fs7, Y_train_fs7)
test_r2_salKNN = knn.score(X_testN_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salKNN)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salKNN)
print()

RMSE = math.sqrt(metrics.mean_squared_error(Y_test_fs7, knn.predict(X_testN)))
print('KNN RMSE:', RMSE)
Expl_Var = metrics.explained_variance_score(Y_test_fs7, knn.predict(X_testN))
print('KNN Explained Variance:', Expl_Var) 

print("CV Runtime:", time.time()-start_ts)

In [0]:
'''#KNN visualizations - plot output

predicted = knn.predict(X_testN)

fig, ax = plt.subplots()
ax.scatter(Y_test, predicted, edgecolors=(0, 0, 0))
ax.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
plt.title('Predicted vs. Actual')
plt.show()
'''

In [0]:
### RANDOM FORESTS ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=4, min_samples_split=2, random_state=12).fit(X_train, Y_train)

train_r2_salaryRF = RF.score(X_train, Y_train)
test_r2_salaryRF = RF.score(X_test, Y_test)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test, RF.predict(X_test)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_salRF = metrics.explained_variance_score(Y_test, RF.predict(X_test))
print('Random Forest Explained Variance on salary data:', ExplVar_salRF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.5979713605726064
COEFFICIENT OF DETERMINATION - Test 0.5956943300767105

Random Forest RMSE on salary data: 1668.7308949704586
Random Forest Explained Variance on salary data: 0.5956955741433703
CV Runtime: 72.76884746551514


In [0]:
#Random forest visualizations - printing results
important_features_dict = {}
features = X_train.columns.tolist()
for x,i in enumerate(RF.feature_importances_):
    important_features_dict[x]=i

# the index of the most important features in decreasing order
lst = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print('Most important features:')
for i in lst:
  print(features[i])

In [0]:
#Random forest plot outputs - salary data
predicted = RF.predict(X_test)

fig, ax = plt.subplots()
ax.scatter(Y_test, predicted, edgecolors=(0, 0, 0))
ax.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
plt.title('Predicted vs. Actual')
plt.show()

In [0]:
from yellowbrick.regressor import ResidualsPlot

visualizer = ResidualsPlot(RF)

visualizer.fit(X_train, Y_train)  # Fit the training data to the model
visualizer.score(X_test, Y_test)  # Evaluate the model on the test data
visualizer.poof()

In [0]:
### GRADIENT BOOSTING ###
print("Gradient Boosting Results\n")

GB = GradientBoostingRegressor(n_estimators=100, loss='ls', learning_rate=0.1, max_depth=3, min_samples_split=3, random_state=88).fit(X_train, Y_train)

train_r_squared = GB.score(X_train, Y_train)
test_r_squared = GB.score(X_test, Y_test)
print("COEFFICIENT OF DETERMINATION - Train", train_r_squared)
print("COEFFICIENT OF DETERMINATION - Test", test_r_squared)
print()

RMSE = math.sqrt(metrics.mean_squared_error(Y_test, GB.predict(X_test)))
print('Gradient Boosting RMSE:', RMSE)
EV = metrics.explained_variance_score(Y_test, GB.predict(X_test))
print('Gradient Boosting Explained Variance:', EV) 

Gradient Boosting Results

COEFFICIENT OF DETERMINATION - Train 0.7395076090923358
COEFFICIENT OF DETERMINATION - Test 0.7306469704392493

Gradient Boosting RMSE: 1362.0477181461608
Gradient Boosting Explained Variance: 0.7306469959158983


In [0]:
#Gradient boosting plot outputs - salary data
predicted = GB.predict(X_test)

fig, ax = plt.subplots()
ax.scatter(Y_test, predicted, edgecolors=(0, 0, 0))
ax.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
plt.title('Predicted vs. Actual')
plt.show()

In [0]:
from yellowbrick.regressor import ResidualsPlot

visualizer = ResidualsPlot(GB)

visualizer.fit(X_train, Y_train)  # Fit the training data to the model
visualizer.score(X_test, Y_test)  # Evaluate the model on the test data
visualizer.poof()

In [0]:
'''### SVM - Either Linear, RBF, Sigmoid, Poly ###
print("SVM Results\n")
SVM = SVR(kernel='linear', gamma='auto', C=1.0).fit(X_train, Y_train)

train_r_squared = SVM.score(X_train, Y_train)
test_r_squared = SVM.score(X_train, Y_train)
print("COEFFICIENT OF DETERMINATION - Train", train_r_squared)
print("COEFFICIENT OF DETERMINATION - Test", test_r_squared)
print()

RMSE = math.sqrt(metrics.mean_squared_error(y_test_scores, SVM.predict(X_test)))
print('SVM RMSE:', RMSE)
Expl_Var = metrics.explained_variance_score(y_test_scores, SVM.predict(X_test))
print('SVM Explained Variance:', Expl_Var) 
'''

In [0]:
### CatBOOST ###

### TEST TRAIN VALIDATION SPLIT ###

num_instances = df_cat.shape[0]

Y = df_cat['AdjSalMonth']
X = df_cat.drop(['AdjSalMonth'], axis=1)

# Creating the train/test split
Y_train_cat, Y_test_cat, X_train_cat, X_test_cat = train_test_split(Y, X, test_size=0.3, random_state=33)  

# Splitting df_train into the train and then holding some out for validation
Y_train_cat, Y_val_cat, X_train_cat, X_val_cat = train_test_split(Y_train_cat, X_train_cat, test_size=0.143, random_state=33)

print("Confirming Splits...")
print("Y_TRAIN_Cat: " + str(Y_train_cat.shape[0]) + " = " + str(round((Y_train_cat.shape[0]/num_instances)*100, 2)) + "%")
print("Y_TEST_cat: " + str(Y_test_cat.shape[0]) + " = " + str(round((Y_test_cat.shape[0]/num_instances)*100, 2)) + "%")
print("Y_VALIDATION_cat: " + str(Y_val_cat.shape[0]) + " = " + str(round((Y_val_cat.shape[0]/num_instances)*100, 2)) + "%")

print()
print("X_TRAIN_cat: " + str(X_train_cat.shape[0]) +  ", " + str(X_train_cat.shape[1]) + " = " + str(round((X_train_cat.shape[0]/num_instances)*100, 2)) + "%")
print("X_TEST_cat: " + str(X_test_cat.shape[0]) +  ", " + str(X_test_cat.shape[1]) + " = " + str(round((X_test_cat.shape[0]/num_instances)*100, 2)) + "%")
print("X_VALIDATION_cat: " + str(X_val_cat.shape[0]) +  ", " + str(X_val_cat.shape[1]) + " = " + str(round((X_val_cat.shape[0]/num_instances)*100, 2)) + "%")

# This is a *close* Train/Test/Validation split of 60/30/10


Confirming Splits...
Y_TRAIN_Cat: 780810 = 59.99%
Y_TEST_cat: 390471 = 30.0%
Y_VALIDATION_cat: 130288 = 10.01%

X_TRAIN_cat: 780810, 21 = 59.99%
X_TEST_cat: 390471, 21 = 30.0%
X_VALIDATION_cat: 130288, 21 = 10.01%


In [0]:
!pip install catboost

     |████████████████████████████████| 61.1MB 1.3MB/s 


In [0]:
from catboost import CatBoostRegressor

print("CatBoost Results\n")
model=CatBoostRegressor(n_estimators=100, random_seed=44, logging_level='Silent')
model.fit(X_train_cat, Y_train_cat, cat_features=([0, 1, 2, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 20]), 
          eval_set=(X_test_cat, Y_test_cat))

RMSE_train = math.sqrt(metrics.mean_squared_error(Y_train_cat, model.predict(X_train_cat)))
print('CatBoost Train RMSE:', RMSE_train)
RMSE_test = math.sqrt(metrics.mean_squared_error(Y_test_cat, model.predict(X_test_cat)))
print('CatBoost Test RMSE:', RMSE_test)
print()
EV_train = metrics.explained_variance_score(Y_train_cat, model.predict(X_train_cat))
print('CatBoost Train Explained Variance:', EV_train) 
EV_test = metrics.explained_variance_score(Y_test_cat, model.predict(X_test_cat))
print('CatBoost Test Explained Variance:', EV_test)

CatBoost Results

CatBoost Train RMSE: 1448.4943788143544
CatBoost Test RMSE: 1453.3227811418521

CatBoost Train Explained Variance: 0.7195983004477134
CatBoost Test Explained Variance: 0.7129062721746708


In [0]:
important_features_dict = {}
features = X_train_cat.columns.tolist()
for x,i in enumerate(model.feature_importances_):
    important_features_dict[x]=i

# the index of the most important features in decreasing order
lst = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print('Most important features:')
for i in lst:
  print(features[i])

In [0]:
predicted = model.predict(X_test_cat)

fig, ax = plt.subplots()
ax.scatter(Y_test_cat, predicted, edgecolors=(0, 0, 0))
ax.plot([Y_test_cat.min(), Y_test_cat.max()], [Y_test_cat.min(), Y_test_cat.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
plt.title('Predicted vs. Actual')
plt.show()

___________________________________________________________________________________________

BEGINNING OF MODELS RUN WITH FIVE FEATURES SELECTED


In [0]:
### LINEAR REGRESSION on FIVE FEATURES###

linear_model = LinearRegression()
linear_model.fit(X_train_fs5,Y_train_fs5)
r_squared = linear_model.score(X_train_fs5, Y_train_fs5)

print("Linear Regression Results\n")
print("COEFFICIENT OF DETERMINATION: ", round(r_squared, 5))

train_r_squared = linear_model.score(X_train_fs5, Y_train_fs5)
test_r_squared = linear_model.score(X_test_fs5, Y_test_fs5)
print("COEFFICIENT OF DETERMINATION - Train: ", train_r_squared)
print("COEFFICIENT OF DETERMINATION - Test: ", test_r_squared)

RMSE = math.sqrt(metrics.mean_squared_error(Y_train_fs5, linear_model.predict(X_train_fs5)))
print("\nLinear Regression RMSE: ", RMSE)
EV = metrics.explained_variance_score(Y_train_fs5, linear_model.predict(X_train_fs5))
print("Linear Regression Explained Variance: ", EV) 

Linear Regression Results

COEFFICIENT OF DETERMINATION:  0.50882
COEFFICIENT OF DETERMINATION - Train:  0.5088204908895909
COEFFICIENT OF DETERMINATION - Test:  0.5197850869371469

Linear Regression RMSE:  1853.769797946865
Linear Regression Explained Variance:  0.508820490889591


In [0]:
### RANDOM FORESTS WITH FIVE FEATURES SELECTED ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=4, min_samples_split=2, random_state=12).fit(X_train_fs5,Y_train_fs5)

train_r2_salaryRF = RF.score(X_train_fs5, Y_train_fs5)
test_r2_salaryRF = RF.score(X_test_fs5, Y_test_fs5)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs5, RF.predict(X_test_fs5)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs5, RF.predict(X_test_fs5))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.5539043914368982
COEFFICIENT OF DETERMINATION - Test 0.5577595682335936

Random Forest RMSE on salary data: 1745.2619217640445
Random Forest Explained Variance on salary data: 0.5577621792934988
CV Runtime: 24.92237687110901


In [0]:
### GRADIENT BOOSTING WITH FIVE FEATURES###
print("Gradient Boosting Results\n")
start_ts=time.time()

GB = GradientBoostingRegressor(n_estimators=100, loss='ls', learning_rate=0.1, max_depth=3, min_samples_split=3, random_state=88).fit(X_train_fs5, Y_train_fs5)

train_r2_GB = GB.score(X_train_fs5, Y_train_fs5)
test_r2_GB = GB.score(X_test_fs5, Y_test_fs5)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_GB)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_GB)
print()

RMSE = math.sqrt(metrics.mean_squared_error(Y_test_fs5, GB.predict(X_test_fs5)))
print('Gradient Boosting RMSE:', RMSE)
EV = metrics.explained_variance_score(Y_test_fs5, GB.predict(X_test_fs5))
print('Gradient Boosting Explained Variance:', EV) 
print("CV Runtime:", time.time()-start_ts)

Gradient Boosting Results

COEFFICIENT OF DETERMINATION - Train 0.6579957349577314
COEFFICIENT OF DETERMINATION - Test 0.6560538728642362

Gradient Boosting RMSE: 1539.1346040729622
Gradient Boosting Explained Variance: 0.6560553753321008
CV Runtime: 49.857590198516846


_____________________

BEGINNING OF MODELS RUN WITH SEVEN FEATURES SELECTED

In [0]:
### LINEAR REGRESSION on SEVEN FEATURES###

linear_model = LinearRegression()
linear_model.fit(X_train_fs7,Y_train_fs7)
r_squared = linear_model.score(X_train_fs7, Y_train_fs7)

print("Linear Regression Results\n")
print("COEFFICIENT OF DETERMINATION: ", round(r_squared, 5))

train_r_squared = linear_model.score(X_train_fs7, Y_train_fs7)
test_r_squared = linear_model.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train: ", train_r_squared)
print("COEFFICIENT OF DETERMINATION - Test: ", test_r_squared)

RMSE = math.sqrt(metrics.mean_squared_error(Y_train_fs7, linear_model.predict(X_train_fs7)))
print("\nLinear Regression RMSE: ", RMSE)
EV = metrics.explained_variance_score(Y_train_fs7, linear_model.predict(X_train_fs7))
print("Linear Regression Explained Variance: ", EV) 

Linear Regression Results

COEFFICIENT OF DETERMINATION:  0.5573
COEFFICIENT OF DETERMINATION - Train:  0.5572997463576648
COEFFICIENT OF DETERMINATION - Test:  0.5648532671418016

Linear Regression RMSE:  1759.9104387953157
Linear Regression Explained Variance:  0.5572997463576648


In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=4, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.5980832520613691
COEFFICIENT OF DETERMINATION - Test 0.5948849144961919

Random Forest RMSE on salary data: 1670.4004503864062
Random Forest Explained Variance on salary data: 0.5948864646277956
CV Runtime: 31.65142250061035


In [0]:
### GRADIENT BOOSTING WITH SEVEN FEATURES###
print("Gradient Boosting Results\n")
start_ts=time.time()

GB = GradientBoostingRegressor(n_estimators=100, loss='ls', learning_rate=0.1, max_depth=3, min_samples_split=3, random_state=88).fit(X_train_fs7, Y_train_fs7)

train_r2_GB = GB.score(X_train_fs7, Y_train_fs7)
test_r2_GB = GB.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_GB)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_GB)
print()

RMSE = math.sqrt(metrics.mean_squared_error(Y_test_fs7, GB.predict(X_test_fs7)))
print('Gradient Boosting RMSE:', RMSE)
EV = metrics.explained_variance_score(Y_test_fs7, GB.predict(X_test_fs7))
print('Gradient Boosting Explained Variance:', EV) 
print("CV Runtime:", time.time()-start_ts)

Gradient Boosting Results

COEFFICIENT OF DETERMINATION - Train 0.733446153413807
COEFFICIENT OF DETERMINATION - Test 0.723919628037696

Gradient Boosting RMSE: 1378.9520231185745
Gradient Boosting Explained Variance: 0.7239198536213511
CV Runtime: 58.83814978599548


_________

BEGINNING OF MODELS WITH EIGHT FEATURES SELECTED

In [0]:
### LINEAR REGRESSION on EIGHT FEATURES###

linear_model = LinearRegression()
linear_model.fit(X_train_fs8,Y_train_fs8)
r_squared = linear_model.score(X_train_fs8, Y_train_fs8)

print("Linear Regression Results\n")
print("COEFFICIENT OF DETERMINATION: ", round(r_squared, 5))

train_r_squared = linear_model.score(X_train_fs8, Y_train_fs8)
test_r_squared = linear_model.score(X_test_fs8, Y_test_fs8)
print("COEFFICIENT OF DETERMINATION - Train: ", train_r_squared)
print("COEFFICIENT OF DETERMINATION - Test: ", test_r_squared)

RMSE = math.sqrt(metrics.mean_squared_error(Y_train_fs8, linear_model.predict(X_train_fs8)))
print("\nLinear Regression RMSE: ", RMSE)
EV = metrics.explained_variance_score(Y_train_fs8, linear_model.predict(X_train_fs8))
print("Linear Regression Explained Variance: ", EV) 

Linear Regression Results

COEFFICIENT OF DETERMINATION:  0.47988
COEFFICIENT OF DETERMINATION - Train:  0.47987732818343076
COEFFICIENT OF DETERMINATION - Test:  0.48526985815314827

Linear Regression RMSE:  1907.6055367614495
Linear Regression Explained Variance:  0.47987732818343076


In [0]:
### RANDOM FORESTS WITH EIGHT FEATURES SELECTED ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=4, min_samples_split=2, random_state=12).fit(X_train_fs8,Y_train_fs8)

train_r2_salaryRF = RF.score(X_train_fs8, Y_train_fs8)
test_r2_salaryRF = RF.score(X_test_fs8, Y_test_fs8)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs8, RF.predict(X_test_fs8)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs8, RF.predict(X_test_fs8))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data



NameError: ignored

In [0]:
### GRADIENT BOOSTING WITH Eight FEATURES###
print("Gradient Boosting Results\n")
start_ts=time.time()

GB = GradientBoostingRegressor(n_estimators=100, loss='ls', learning_rate=0.1, max_depth=3, min_samples_split=3, random_state=88).fit(X_train_fs8, Y_train_fs8)

train_r2_GB = GB.score(X_train_fs8, Y_train_fs8)
test_r2_GB = GB.score(X_test_fs8, Y_test_fs8)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_GB)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_GB)
print()

RMSE = math.sqrt(metrics.mean_squared_error(Y_test_fs8, GB.predict(X_test_fs8)))
print('Gradient Boosting RMSE:', RMSE)
EV = metrics.explained_variance_score(Y_test_fs8, GB.predict(X_test_fs8))
print('Gradient Boosting Explained Variance:', EV) 
print("CV Runtime:", time.time()-start_ts)

____________________
PARAMETER OPTIMIZATION

In [0]:
### TEST TRAIN VALIDATION SPLIT with FS###

num_instances = df_num.shape[0]

Y_FS = df_num['AdjSalMonth']
X_FS = df_num[['months_employed', 'fte', 'dist_exp', 'state_exp', 'first_3', 'grade_taught', 'Bachelors']]

# Creating the train/test split
Y_FS_train, Y_FS_test, X_FS_train, X_FS_test = train_test_split(Y_FS, X_FS, test_size=0.3, random_state=33)  

# Splitting df_train into the train and then holding some out for validation
Y_FS_train, Y_FS_val, X_FS_train, X_FS_val = train_test_split(Y_FS_train, X_FS_train, test_size=0.143, random_state=33)

print("Confirming Splits...")
print("Y_FS_TRAIN: " + str(Y_FS_train.shape[0]) + " = " + str(round((Y_FS_train.shape[0]/num_instances)*100, 2)) + "%")
print("Y_FS_TEST: " + str(Y_FS_test.shape[0]) + " = " + str(round((Y_FS_test.shape[0]/num_instances)*100, 2)) + "%")
print("Y_FS_VALIDATION: " + str(Y_FS_val.shape[0]) + " = " + str(round((Y_FS_val.shape[0]/num_instances)*100, 2)) + "%")

print()
print("X_FS_TRAIN: " + str(X_FS_train.shape[0]) +  ", " + str(X_FS_train.shape[1]) + " = " + str(round((X_FS_train.shape[0]/num_instances)*100, 2)) + "%")
print("X_FS_TEST: " + str(X_FS_test.shape[0]) +  ", " + str(X_FS_test.shape[1]) + " = " + str(round((X_FS_test.shape[0]/num_instances)*100, 2)) + "%")
print("X_FS_VALIDATION: " + str(X_FS_val.shape[0]) +  ", " + str(X_FS_val.shape[1]) + " = " + str(round((X_FS_val.shape[0]/num_instances)*100, 2)) + "%")

In [0]:
### LINEAR REGRESSION ###

linear_model = LinearRegression()
linear_model.fit(X_FS_train,Y_FS_train)
r_squared = linear_model.score(X_FS_train, Y_FS_train)

print("Linear Regression Results\n")
print("COEFFICIENT OF DETERMINATION: ", round(r_squared, 5))

train_FS_r_squared = linear_model.score(X_FS_train, Y_FS_train)
test_FS_r_squared = linear_model.score(X_FS_test, Y_FS_test)
print("COEFFICIENT OF DETERMINATION - Train: ", train_FS_r_squared)
print("COEFFICIENT OF DETERMINATION - Test: ", test_FS_r_squared)

RMSE = math.sqrt(metrics.mean_squared_error(Y_FS_train, linear_model.predict(X_FS_train)))
print("\nLinear Regression RMSE: ", RMSE)
EV = metrics.explained_variance_score(Y_FS_train, linear_model.predict(X_FS_train))
print("Linear Regression Explained Variance: ", EV) 

Linear Regression Results

COEFFICIENT OF DETERMINATION:  0.5573
COEFFICIENT OF DETERMINATION - Train:  0.5572997463576648
COEFFICIENT OF DETERMINATION - Test:  0.5648532671418016

Linear Regression RMSE:  1759.9104387953157
Linear Regression Explained Variance:  0.5572997463576648


In [0]:
### Decision Tree - All Depth###
from sklearn.tree import DecisionTreeRegressor
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_FS_train,Y_FS_train)
r_squared = decision_tree.score(X_FS_train, Y_FS_train)

print("Decision Tree Results\n")
print("COEFFICIENT OF DETERMINATION: ", round(r_squared, 5))

train_FS_r_squared = decision_tree.score(X_FS_train, Y_FS_train)
test_FS_r_squared = decision_tree.score(X_FS_test, Y_FS_test)
print("COEFFICIENT OF DETERMINATION - Train: ", train_FS_r_squared)
print("COEFFICIENT OF DETERMINATION - Test: ", test_FS_r_squared)

RMSE = math.sqrt(metrics.mean_squared_error(Y_FS_train, decision_tree.predict(X_FS_train)))
print("\nDecision Tree RMSE: ", RMSE)
EV = metrics.explained_variance_score(Y_FS_train, decision_tree.predict(X_FS_train))
print("Decision Tree Explained Variance: ", EV) 

Decision Tree Results

COEFFICIENT OF DETERMINATION:  0.85251
COEFFICIENT OF DETERMINATION - Train:  0.8525115316308082
COEFFICIENT OF DETERMINATION - Test:  0.6770750887325399

Decision Tree RMSE:  1015.8152178839136
Decision Tree Explained Variance:  0.8525115316308083


In [0]:
### RANDOM FORESTS ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=4, min_samples_split=2, random_state=12).fit(X_FS_train, Y_FS_train)

train_FS_r2_salaryRF = RF.score(X_FS_train, Y_FS_train)
test_FS_r2_salaryRF = RF.score(X_FS_test, Y_FS_test)
print("COEFFICIENT OF DETERMINATION - Train", train_FS_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_FS_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_FS_test, RF.predict(X_FS_test)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_salRF = metrics.explained_variance_score(Y_FS_test, RF.predict(X_FS_test))
print('Random Forest Explained Variance on salary data:', ExplVar_salRF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.5980832520613691
COEFFICIENT OF DETERMINATION - Test 0.5948849144961919

Random Forest RMSE on salary data: 1670.4004503864062
Random Forest Explained Variance on salary data: 0.5948864646277956
CV Runtime: 28.798956871032715


In [0]:
### GRADIENT BOOSTING ###
print("Gradient Boosting Results\n")
start_ts=time.time()

GB = GradientBoostingRegressor(n_estimators=100, loss='ls', learning_rate=0.1, max_depth=3, min_samples_split=3, random_state=88).fit(X_FS_train, Y_FS_train)

train_FS_r_squared = GB.score(X_FS_train, Y_FS_train)
test_FS_r_squared = GB.score(X_FS_test, Y_FS_test)
print("COEFFICIENT OF DETERMINATION - Train", train_FS_r_squared)
print("COEFFICIENT OF DETERMINATION - Test", test_FS_r_squared)
print()

RMSE = math.sqrt(metrics.mean_squared_error(Y_FS_test, GB.predict(X_FS_test)))
print('Gradient Boosting RMSE:', RMSE)
EV = metrics.explained_variance_score(Y_FS_test, GB.predict(X_FS_test))
print('Gradient Boosting Explained Variance:', EV) 
print("CV Runtime:", time.time()-start_ts)

Gradient Boosting Results

COEFFICIENT OF DETERMINATION - Train 0.733446153413807
COEFFICIENT OF DETERMINATION - Test 0.723919628037696

Gradient Boosting RMSE: 1378.9520231185745
Gradient Boosting Explained Variance: 0.7239198536213511
CV Runtime: 52.55981993675232


In [0]:
### GRADIENT BOOSTING-Parameter Testing###
print("Gradient Boosting Results\n")
start_ts=time.time()
GB = GradientBoostingRegressor(n_estimators=300, loss='ls', learning_rate=0.5, max_depth=3, min_samples_split=10, random_state=88).fit(X_FS_train, Y_FS_train)

train_FS_r_squared = GB.score(X_FS_train, Y_FS_train)
test_FS_r_squared = GB.score(X_FS_test, Y_FS_test)
val_FS_r_squared = GB.score(X_FS_val, Y_FS_val)
print("COEFFICIENT OF DETERMINATION - Train", train_FS_r_squared)
print("COEFFICIENT OF DETERMINATION - Test", test_FS_r_squared)
print("COEFFICIENT OF DETERMINATION - Validation", val_FS_r_squared)
print()

RMSE = math.sqrt(metrics.mean_squared_error(Y_FS_test, GB.predict(X_FS_test)))
print('Gradient Boosting RMSE:', RMSE)
EV = metrics.explained_variance_score(Y_FS_test, GB.predict(X_FS_test))
print('Gradient Boosting Explained Variance:', EV) 
print("CV Runtime:", time.time()-start_ts)

Gradient Boosting Results

COEFFICIENT OF DETERMINATION - Train 0.7750778077585099
COEFFICIENT OF DETERMINATION - Test 0.7522141389638844
COEFFICIENT OF DETERMINATION - Validation 0.7486950665381664

Gradient Boosting RMSE: 1306.3803861472109
Gradient Boosting Explained Variance: 0.7522142999242081
CV Runtime: 145.85729551315308


_____________________

PARAMETER OPTIMIZATION FOR RANDOM FOREST

In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED - MSE criterion ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=4, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.5980832520613691
COEFFICIENT OF DETERMINATION - Test 0.5948849144961919

Random Forest RMSE on salary data: 1670.4004503864062
Random Forest Explained Variance on salary data: 0.5948864646277956
CV Runtime: 36.8925940990448


In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED - MAE criterion - does not run###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mae', max_features=.33, max_depth=4, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data



In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED w/ 3 max_depth ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=3, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.5382492792685966
COEFFICIENT OF DETERMINATION - Test 0.5375339656789401

Random Forest RMSE on salary data: 1784.7250175468744
Random Forest Explained Variance on salary data: 0.537535952678613
CV Runtime: 28.61750817298889


In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED w/ 5 max depth ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=5, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.6409346428045845
COEFFICIENT OF DETERMINATION - Test 0.6358123590621114

Random Forest RMSE on salary data: 1583.7768564936941
Random Forest Explained Variance on salary data: 0.635813655130471
CV Runtime: 44.286580324172974


In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED w/ 2 max depth ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=2, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.4443478401093631
COEFFICIENT OF DETERMINATION - Test 0.44660705399643064

Random Forest RMSE on salary data: 1952.30739864316
Random Forest Explained Variance on salary data: 0.44661049150428944
CV Runtime: 20.476091384887695


In [0]:

### RANDOM FORESTS WITH SEVEN FEATURES SELECTED w/ 6 max depth ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=6, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.6732013370390858
COEFFICIENT OF DETERMINATION - Test 0.6649871574951786

Random Forest RMSE on salary data: 1519.015196067169
Random Forest Explained Variance on salary data: 0.664988319021697
CV Runtime: 52.01949501037598


In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED w/ 7 max depth ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=7, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.6994901604341252
COEFFICIENT OF DETERMINATION - Test 0.6867625852885058

Random Forest RMSE on salary data: 1468.8187441309665
Random Forest Explained Variance on salary data: 0.6867634731496548
CV Runtime: 57.81753945350647


In [0]:
### RANDOM FORESTS WITH FIVE FEATURES SELECTED w/ adjusting max features paramter ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=5, max_features = 0.25, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.6022346894654216
COEFFICIENT OF DETERMINATION - Test 0.5998627372824186

Random Forest RMSE on salary data: 1660.106267048145
Random Forest Explained Variance on salary data: 0.5998643519688537
CV Runtime: 33.69183015823364


In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED w/ min samples at 3 ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=5, min_samples_split=3, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.6409064723515681
COEFFICIENT OF DETERMINATION - Test 0.6357298339623736

Random Forest RMSE on salary data: 1583.956288639304
Random Forest Explained Variance on salary data: 0.6357311007722777
CV Runtime: 44.407559633255005


In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED w/ min samples at 4 ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=100, criterion='mse', max_features=.33, max_depth=5, min_samples_split=4, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.6401553892041767
COEFFICIENT OF DETERMINATION - Test 0.6350513623311467

Random Forest RMSE on salary data: 1585.4307018166935
Random Forest Explained Variance on salary data: 0.6350526335932669
CV Runtime: 43.75077271461487


In [0]:
### RANDOM FORESTS WITH SEVEN FEATURES SELECTED - final model ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=50, criterion='mse', max_features=.33, max_depth=5, min_samples_split=2, random_state=12).fit(X_train_fs7,Y_train_fs7)

train_r2_salaryRF = RF.score(X_train_fs7, Y_train_fs7)
test_r2_salaryRF = RF.score(X_test_fs7, Y_test_fs7)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test_fs7, RF.predict(X_test_fs7)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test_fs7, RF.predict(X_test_fs7))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.6409194115280918
COEFFICIENT OF DETERMINATION - Test 0.6356874921272451

Random Forest RMSE on salary data: 1584.048343484098
Random Forest Explained Variance on salary data: 0.6356889322292254
CV Runtime: 23.183361291885376


In [0]:
### RANDOM FORESTS WITH NO FEATURE SELECTION - final model ###
print('Random Forest Results on teacher salary data')
print()
start_ts=time.time()
RF = RandomForestRegressor(n_estimators=50, criterion='mse', max_features=.33, max_depth=5, min_samples_split=2, random_state=12).fit(X_train,Y_train)

train_r2_salaryRF = RF.score(X_train, Y_train)
test_r2_salaryRF = RF.score(X_test, Y_test)
print("COEFFICIENT OF DETERMINATION - Train", train_r2_salaryRF)
print("COEFFICIENT OF DETERMINATION - Test", test_r2_salaryRF)
print()

RMSE_salRF = math.sqrt(metrics.mean_squared_error(Y_test, RF.predict(X_test)))
print('Random Forest RMSE on salary data:', RMSE_salRF)
ExplVar_RF = metrics.explained_variance_score(Y_test, RF.predict(X_test))
print('Random Forest Explained Variance on salary data:', ExplVar_RF) 

print("CV Runtime:", time.time()-start_ts)

Random Forest Results on teacher salary data

COEFFICIENT OF DETERMINATION - Train 0.6413960041875761
COEFFICIENT OF DETERMINATION - Test 0.6358898800136852

Random Forest RMSE on salary data: 1583.6082862164444
Random Forest Explained Variance on salary data: 0.6358904766741384
CV Runtime: 44.42801237106323
